# QISKIT Lab 1 - Quantum States and Operations on 1-qubit

1. S. G. Nana Engo, serge.nana-engo@facsciences-uy1.cm
    * Department of Physics, Faculty of Science, University of Yaounde I
1. J-P. Tchapet Njafa, jean-pierre.tchapet-njafa@univ-maroua.cm
    * Department of Physics, Faculty of Science, University of Maroua
1. P. Djorwe, djorwepp@gmail.com
    * Department of Physics, Faculty of Science, University of Ngaoundere
       
*Adaptation of the __Qiskit Global Summer School 2020.__*

January 2023

*Prérequis cours*
- Chapitre 1 Systèmes Multi-qubits
- Chapitre 2 Portes Quantiques

*Autres documents pertinents (web)*
- [Access IBM Quantum Systems](https://qiskit.org/documentation/install.html#access-ibm-quantum-systems)
- [IBM Quantum Systems Configuration](https://quantum-computing.ibm.com/docs/manage/backends/configuration)
- [Transpile](https://qiskit.org/documentation/apidoc/transpiler.html)
- [IBM Quantum account](https://quantum-computing.ibm.com/docs/manage/account/ibmq)
- [Quantum Circuits](https://qiskit.org/documentation/apidoc/circuit.html)  

# Installation

[Qiskit](https://qiskit.org/documentation/) est un kit de développement logiciel (SDK) open source permettant de travailler avec des calculateurs quantiques au niveau des impulsions, des circuits et des algorithmes. Qiskit accélère le développement d'applications quantiques en fournissant un ensemble complet d'outils nécessaires pour interagir avec les systèmes et simulateurs quantiques.

## Création d'environnement python dédié

Il est conseillé de créer un environnement python dédié à qiskit (par exemple `qiskit-env`). Ceci suppose qu'une distribution python est déja installé sur votre système d'exploitation. Nous vous conseillons vivement [miniconda](https://repo.anaconda.com/miniconda).

> `conda create --name qiskit-env python -y`

## Installation de qiskit

* Dans un terminal, entrer

> `pip install qiskit[visualization]`

* Dans une cellule Jupyter Notebook ou jupyter lab, entrer et exécuter

> `!pip install qiskit[visualization]`


## Vérification version installée

Pour vérifier la version installée de Qiskit , entrer dans une cellule et exécuter

In [1]:
import qiskit
qiskit.__version__

'0.22.4'

* Pour voir les versions des modules Qiskit installés. _Noter qu'en programmation, les instructions et/ou les résultats du code dépendent des versions des modules_.

In [ ]:
qiskit.__qiskit_version__

In [5]:
import qiskit.tools.jupyter #only in jupyter notebook
%qiskit_version_table

<div class="alert alert-warning"> Dans un Jupyter Notebook,
    <ul>
        <li>pour obtenir de l'aide sur une fonction, placez le curseur de texte sur la fonction et appuyez sur <code>Shift + Tab</code></li>
        <li>pour exécuter une cellule, appuyez sur <code>Shift + Return / Enter</code>.</li>
    </ul>
</div>

## Remarque importante - Qubit le moins significatif



<div class="alert alert-info">
Dans les manuels de physique (et dans nos notes de cours), un système multi-qubit de qubits $n$ est écrit $|j_0j_1...j_{n-1}\rangle$, donc le produit tenseur des opérateurs est $Q_0\otimes Q_1\otimes ...\otimes Q_{n-1} $. Cela signifie que le qubit (opérateur) le moins significatif est à l'extrême gauche et le plus à l'extrême droite. Mais la représentation dans Qiskit est le contraire, c'est-à-dire:
\begin{align*}
& |j_{n-1}...j_1j_0\rangle&&\text{ pour un système de $n$-qubits}\\
&Q_{n-1}\otimes\cdots\otimes Q_1\otimes Q_0&&\text{ pour un produit tensoriel} 
\end{align*}
En d'autres termes, dans Qiskit, $|01\rangle\equiv\underset{\text{qubit 1}}{|0\rangle}\ \underset{\text{qubit 0}}{|1\rangle}$, qui est à l'opposé de ce que nous utilisons dans nos notes de cours, $|01\rangle\equiv\underset{\text{qubit 0}}{|0\rangle}\ \underset{\text{qubit 1}}{|1\rangle}$. <br>
    <b>Nous adoptons cette notation dans les développements analytiques pour des besoins de coherence avec les résulats des simulations.</b>
</div>

# États et opérateurs à 1-qubit

Dans ce Lab, nous allons apprendre à écrire du code `Qiskit` pour étudier les états à un et à plusieurs qubits.

*Nous rappelons qu'un seul état quantique de qubit s'écrit plus généralement comme:
\begin{equation*}
|\psi\rangle=\cos\frac{\theta}{2}|0\rangle+\sin\frac{\theta}{2}e^{i\phi}|1\rangle
=\sqrt{p}|0\rangle + \sqrt{1-p}e^{i\phi}|0\rangle,
\end{equation*}
avec $|\cos\frac{\theta}{2}|^2+|\sin\frac{\theta}{2}e^{i\phi}|^2=1$, où $0\leq\phi<2\pi$ et $0\leq\theta\leq\pi$. Les portes à 1-qubit peuvent ensuite être utilisées pour manipuler l'état quantique $|\psi\rangle$ en changeant $\theta$, $\phi$ ou les deux.* 


## Workflow étape par étape

1. Importer des packages

2. Initialiser les variables

3. Ajouter des portes

4. Visualiser le circuit

5. Simuler l'expérience ou faire évoluer le système

6. Visualiser les résultats

## Importer les packages nécessaires

In [ ]:
from qiskit import QuantumCircuit 
from qiskit.quantum_info import Statevector
from qiskit.quantum_info.operators import Operator, Pauli
from qiskit.visualization import array_to_latex, plot_bloch_multivector, plot_histogram, visualize_transition 

import numpy as np


Les modules importés sont:

- `QuantumCircuit` qui peut être considéré comme les instructions du système quantique. Il contient toutes vos opérations quantiques;

- `Statevector` qui est la classe des objets sous forme de vecteur d'état; 

- `Operator` qui est la classe des opérateurs sous forme de matrice;

- `Pauli` qui est la classe de matrices de Pauli;

- `array _to_ latex` qui écrit les tableaux (matrices) sous forme de LaTeX;

- `plot_bloch_multivector` qui permet de visualiser un vecteur d'état de type `array` sur une sphère de Bloch

- `visualize_transition` qui crée une animation montrant les transitions entre les états d'un 1-qubit en appliquant des portes quantiques;

- `numpy` pour les calculs numériques en python.


Qiskit est trop généreusement avec la précision numérique, que nous supprimons pour une meilleure lisibilité de la sortie:

In [ ]:
np.set_printoptions(precision=4, suppress=True)

## Initialiser un vecteur d'état dans le circuit

Créons les vecteurs d'état $|0\rangle = \begin{pmatrix}1\\0\end{pmatrix}$ et $|1\rangle = \begin{pmatrix}0\\1\end{pmatrix}$:

In [ ]:
sv0 = Statevector.from_label('0')
sv1 = Statevector.from_label('1')

### Représentation matricielle

Nous pouvons voir ce que contient l'objet `sv`:

In [ ]:
sv0

Le vecteur lui-même peut être trouvé en écrivant

In [ ]:
sv0.data

Pour visualiser le vecteur en LaTeX

In [ ]:
sv0.draw('latex')

In [ ]:
sv1.draw('latex')

### Visualiser l'état

Nous pouvons visualiser géométrique cet état 1-qubit en utilisant l'argument `bloch` de la fonction de visualisation `draw`. 

On rappelle que vecteur Bloch pour un 1-qubit est défini comme
\begin{align}
\left[b_x=\mathtt{Tr}(\mathtt{X}\rho),b_y=\mathtt{Tr}(\mathtt{Y}\rho),z=\mathtt{Tr}(\mathtt{Z}\rho)\right]
\end{align}

In [ ]:
sv0.draw('bloch')

In [ ]:
sv1.draw('bloch')

### Probabilité  d'obtenir un état

- Un moyen simple d'obtenir des probabilités, à partir du vecteur d'état,de chaque chaîne de bits de la base de calcul est

In [ ]:
sv0.probabilities()

In [ ]:
array_to_latex(sv0.probabilities())

In [ ]:
(sv0).probabilities_dict()

In [ ]:
sv0_counts = sv0.sample_counts(shots=1000)

plot_histogram(sv0_counts)

Si le système est initialement dans l'état $|0\rangle$ et qu'il n'évolue pas,  apres la mesure, la probabilité de le trouver dans le meme état $|0\rangle$ est 1, et 0 dans l'état $|1\rangle$.

In [ ]:
sv1.probabilities()

In [ ]:
array_to_latex(sv1.probabilities())

In [ ]:
(sv1).probabilities_dict()

In [ ]:
sv1_counts = sv1.sample_counts(shots=1000)

plot_histogram(sv1_counts)

Si le système est initialement dans l'état $|1\rangle$ et qu'il n'évolue pas,  apres la mesure, la probabilité de le trouver dans le meme état $|1\rangle$ est 1, et 0 dans l'état $|0\rangle$.

## Initialiser un circuit quantique

La syntaxe

> `QuantumCircuit (int _qb, int_ cb)`

permet d'initialiser un circuit quantique avec `int _qb` qubits et` int_ cb` le nombre bits classiques.

In [ ]:
mycircuit = QuantumCircuit(1) 

Dessiner le circuit avec `mpl`=matplotlib : image avec couleur rendue uniquement en Python

In [ ]:
mycircuit.draw('mpl',initial_state=True)

Le circuit quantique ci-dessus ne contient aucune porte. Par conséquent, si nous commençons dans n'importe quel état, disons $|0\rangle$, l'application de ce circuit à l'état ne change pas l'état.

In [ ]:
plot_bloch_multivector(mycircuit)

## Appliquer l'état au circuit

Nous pouvons maintenant appliquer le circuit quantique `mycircuit` à cet état en utilisant ce qui suit:
<!-- \begin{equation} -->
$$\mathbb{I}|0\rangle=|0\rangle$$
<!-- \end{equation} -->

In [ ]:
new_sv0 = sv0.evolve(mycircuit)

Une fois de plus, nous pouvons visualiser le nouveau statevector en écrivant

In [ ]:
new_sv0

In [ ]:
new_sv0.draw('latex', prefix='\mathbb{I}|0\\rangle=')

In [ ]:
new_sv0.draw('bloch')

Comme on peut le voir, le statevector n'a pas changé puisqu'il n'a été soumis à aucun opérateur (**le calcul quatique est une évolution unitaire!).

* Une manière directe d'appliquer une porte ou un opérateur à un état est d'utiliser `Statevector.evolve (Operator)`. Par example,

In [ ]:
new_sv0a = sv0.evolve(Operator(Pauli('I')))
new_sv0a.draw('latex', prefix='\mathbb{I}|0\\rangle=')

* L'application de la porte d'identité équivaut à laisser le fil vide.

In [ ]:
mycircuit.id(0)
mycircuit.draw('mpl', initial_state=True)

<div class="alert alert-info"> L'index des fils d'un circuit quantique commence par $0$. Donc, au lieu de $1$ pour étiqueter le premier qubit, on utilise $0$.</div>

In [ ]:
array_to_latex(Operator(mycircuit), prefix='\mathbb{I}=')

In [ ]:
(sv0.evolve(mycircuit)).draw('latex', prefix='\mathbb{I}|0\\rangle=')

## Evaluer la fidélité d'un état

La **fidélité** est utile pour vérifier si deux états sont identiques ou non. Pour les états quantiques (purs) $|\psi_1\rangle$ et $|\psi_2\rangle$, la fidélité est
\begin{equation}
 \mathcal{F}(|\psi_1\rangle,|\psi_2\rangle)=|\langle\psi_1|\psi_2\rangle|^2 .
\end{equation}

Rappelez-vous le concept de la projection d'état vu dans les notes de cours. Nous pouvons calculer la projection de `new_sv0` sur` sv0` en écrivant

In [ ]:
from qiskit.quantum_info import state_fidelity

state_fidelity(sv0, new_sv0)

## Ajouter une porte X et évaluer la fidélité

Comme nous le savons, l'application d'une porte $\mathtt{X}$ fait basculer le qubit de l'état $|k\rangle$ à l'état $|1-k\rangle$ (inversion ou flip du bit),
\begin{align}
&\mathtt{X}|k\rangle =|1-k\rangle,  
&&\mathtt{X}|0\rangle =|1\rangle ,
&\mathtt{X}|1\rangle =|0\rangle.  
\end{align}
Pour voir cela clairement, nous allons d'abord créer un circuit quantique à un qubit avec la porte $\mathtt{X}$.

### Méthode 1 - Statevector

In [ ]:
X = Operator(Pauli('X'))

In [ ]:
array_to_latex(X, prefix='\mathtt{X}=')

In [ ]:
array_to_latex(X&X, prefix='\mathtt{X}^2=')

In [ ]:
(sv0.evolve(X)).draw('latex', prefix='\mathtt{X}|0\\rangle=')

In [ ]:
(sv1.evolve(X)).draw('latex', prefix='\mathtt{X}|1\\rangle=')

### Methode 2 - QuantumCircuit

In [ ]:
mycircuit1 = QuantumCircuit(1)
mycircuit1.x(0)

mycircuit1.draw('mpl',initial_state=True)

In [ ]:
array_to_latex(Operator(mycircuit1), prefix='\mathtt{X} = ')

* Maintenant, nous pouvons appliquer ce circuit sur notre état en écrivant

In [ ]:
svx0 = sv0.evolve(mycircuit1)
svx0.draw('latex', prefix='\mathtt{X}|0\\rangle=')

Comme on peut le voir, le statevector correspond maintenant à celui de l'état $|1\rangle= \begin{pmatrix}0\\1\end{pmatrix}$.

In [ ]:
svx0.draw('bloch')

* Maintenant, la projection de `svx0` sur` sv0` est

In [ ]:
state_fidelity(svx0, sv0)

Cela n'est pas surprenant car les états $|0\rangle$ et $|1\rangle$ sont orthogonaux. Par conséquent, $\langle0|1\rangle = 0$.

### Visualisation animé de l'action de X 

In [ ]:
visualize_transition(mycircuit1)

###  Exercice Portes de Pauli Y et Z

Appliquer la toute procédure précédente avec les portes $\mathtt{Y}$ et $\mathtt{Z}$.

## État superposé

Nous pouvons créer des états superposés avec la porte Hadamard $\mathtt{H}$:

\begin{align}
&\mathtt{H}|x\rangle=\frac{1}{\sqrt{2}}\sum_y(-1)^{xy} |y\rangle,
&&|+\rangle = \mathtt{H}|0\rangle =\frac{1}{\sqrt{2}}(|0\rangle+|1\rangle ),
&|-\rangle = \mathtt{H}|1\rangle =\frac{1}{\sqrt{2}}(|0\rangle-|1\rangle ).
\end{align}

### Methode 1 - Statevector

In [ ]:
Z = Operator(Pauli('Z'))
H = (X + Z)/np.sqrt(2)
array_to_latex(H, prefix='\mathtt{H}=') 

#### Application de H et visualisation sur la sphère de Bloch

* $|+\rangle = \mathtt{H}|0\rangle$

In [ ]:
(sv0.evolve(H)).draw('latex', prefix='\mathtt{H}|0\\rangle=|+\\rangle=')

In [ ]:
(sv0.evolve(H)).draw('bloch')

* $|-\rangle=\mathtt{H}|1\rangle$

In [ ]:
(sv1.evolve(H)).draw('latex', prefix='\mathtt{H}|1\\rangle=|-\\rangle=')

In [ ]:
(sv1.evolve(H)).draw('bloch')

#### Probabilités

In [ ]:
(sv0.evolve(H)).probabilities()

In [ ]:
(sv1.evolve(H)).probabilities()

In [ ]:
svh0_counts = (sv0.evolve(H)).sample_counts(shots=1000)

plot_histogram(svh0_counts)

In [ ]:
(sv0.evolve(H)).probabilities_dict()

In [ ]:
svh1_counts = (sv1.evolve(H)).sample_counts(shots=1000)

plot_histogram(svh1_counts)

Si le système est initialement dans l'état $|0\rangle$ ou $|0\rangle$ et qu'il évolue sous l'action de $\mathtt{H}$,  apres la mesure, on a une probabilité $\frac12$ de le trouver dans l'état $|0\rangle$ ou dans l'état $|1\rangle$.

### Methode 2 - QuantumCircuit

In [ ]:
mycircuit2 = QuantumCircuit(1)
mycircuit2.h(0)
mycircuit2.draw('mpl',initial_state=True)

In [ ]:
array_to_latex(Operator(mycircuit2), prefix='\mathtt{H}=')

####  Ajoutez le $|0\rangle$ au circuit.

In [ ]:
svh0 = sv0.evolve(mycircuit2)
svh0.draw('latex', prefix='\mathtt{H}|0\\rangle=|+\\rangle=')

In [ ]:
svh0.draw('bloch')

#### Ajoutez le $|1\rangle$ au circuit

In [ ]:
svh1 = sv1.evolve(mycircuit2)
svh1.draw('latex', prefix='\mathtt{H}|1\\rangle=|-\\rangle=')

In [ ]:
svh1.draw('bloch')

### Visualisation animé de l'action de H 

In [ ]:
visualize_transition(mycircuit2)

### Exercices

### Visualiser un 1-qubit

Utiliser `plot_bloch_mutivector()` pour visualiser les états 1-qubit:
\begin{align}
&|0\rangle, && |1\rangle, && |+\rangle, &&|-\rangle, && |r\rangle,&|l\rangle.   
\end{align}

### Etats propres de X

Vérifier que $|\pm\rangle$ sont des états propres de $\mathtt{X}$.

### Opérateurs de translation

En utilisant les opérateurs Pauli comme des  Qiskit Operator class, par exemple, `Z = Operator(Pauli('Z'))`, et en tenant compte du fait que l'opérateur Hadamard $\mathtt{H}=\frac{1}{\sqrt{2}}(\mathtt{X}+\mathtt{Z})$, montrer que
\begin{align}
& \mathtt{H}^2=\mathbb{I}, && \mathtt{H}\mathtt{Y}\mathtt{H}=-\mathtt{Y},
&& \mathtt{H}\mathtt{X}\mathtt{H}=\mathtt{Z},
& \mathtt{H}\mathtt{Z}\mathtt{H}=\mathtt{X}.
\end{align}

<!-- ### Probabilité d'un 1 qubit

1. Si un état initial est $|+\rangle$, quelle est la probabilité de le mesurer dans l'état $|+\rangle$?
2. Utilisez Qiskit pour afficher la probabilité de mesurer un qubit $|0\rangle$ dans les états $|\pm\rangle$? -->